## Jumpstart OD finetuning

- Jumpstart를 활용하여 pre-trained 된 모델을 곧바로 배포하거나, fine-tuning하여 활용할 수 있습니다.
- [코드 예시](https://github.com/aws/amazon-sagemaker-examples/blob/main/introduction_to_amazon_algorithms/jumpstart_object_detection/Amazon_JumpStart_Object_Detection.ipynb)

In [ ]:
!pip install sagemaker ipywidgets --upgrade --quiet

In [ ]:
import sagemaker, boto3, json
from sagemaker import get_execution_role

aws_role = get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()
bucket = sess.default_bucket()

In [ ]:
import IPython
from ipywidgets import Dropdown

# download JumpStart model_manifest file.
boto3.client("s3").download_file(
    f"jumpstart-cache-prod-{aws_region}", "models_manifest.json", "models_manifest.json"
)
with open("models_manifest.json", "rb") as json_file:
    model_list = json.load(json_file)

# filter-out all the Object Detection models from the manifest list.
od_models = []
for model in model_list:
    model_id = model["model_id"]
    if ("-od-" in model_id or "-od1-" in model_id) and model_id not in od_models:
        od_models.append(model_id)

print(f"Number of available models: {len(model_list)}")        
print(f"Number of models available for object detection inference: {len(od_models)}")

# display the model-ids in a dropdown to select a model for inference.
infer_model_dropdown = Dropdown(
    options=od_models,
    value="pytorch-od-nvidia-ssd",
    description="Select a model:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)


In [ ]:

display(IPython.display.Markdown("### 아래에서 사용할 pre-trained 모델을 선택해 주세요"))
display(infer_model_dropdown)

In [ ]:
print(infer_model_dropdown.value)

In [ ]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base

# model_version="*" fetches the latest version of the model
infer_model_id, infer_model_version = infer_model_dropdown.value, "*"

endpoint_name = name_from_base(f"jumpstart-object-detection-infer-{infer_model_id}")

# inference_instance_type = "ml.m5.2xlarge"  # cpu 모델 배포시 request 부분도 그에 맞도록 변경
inference_instance_type = "ml.g4dn.2xlarge"

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    image_scope="inference",
    model_id=infer_model_id,
    model_version=infer_model_version,
    instance_type=inference_instance_type,
)

# Retrieve the inference script uri. This includes scripts for model loading, inference handling etc.
deploy_source_uri = script_uris.retrieve(
    model_id=infer_model_id, model_version=infer_model_version, script_scope="inference"
)

# Retrieve the base model uri
base_model_uri = model_uris.retrieve(
    model_id=infer_model_id, model_version=infer_model_version, model_scope="inference"
)

# Create the SageMaker model instance
model = Model(
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    model_data=base_model_uri,
    entry_point="inference.py",  # entry point file in source_dir and present in deploy_source_uri
    role=aws_role,
    predictor_cls=Predictor,
    name=endpoint_name,
)

# deploy the Model. Note that we need to pass Predictor class when we deploy model through Model class,
# for being able to run inference through the sagemaker API.
base_model_predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    predictor_cls=Predictor,
    endpoint_name=endpoint_name,
)

In [ ]:
import json

def query(model_predictor, image_file_name):

    with open(image_file_name, "rb") as file:
        input_img_rb = file.read()

    query_response = model_predictor.predict(
        input_img_rb,
        {
            "ContentType": "application/x-image",
            "Accept": "application/json;verbose;n_predictions=5",
            # "Accept": "application/json;verbose",
        },
    )
    return query_response

def parse_response(query_response):
    model_predictions = json.loads(query_response)
    normalized_boxes, classes, scores, labels = (
        model_predictions["normalized_boxes"],
        model_predictions["classes"],
        model_predictions["scores"],
        model_predictions["labels"],
    )
    # Substitute the classes index with the classes name
    class_names = [labels[int(idx)] for idx in classes]
    return normalized_boxes, class_names, scores

In [ ]:
import matplotlib.patches as patches
from matplotlib import pyplot as plt
from PIL import Image
from PIL import ImageColor
import numpy as np

conf_threshold = 0.25

def display_predictions(img_jpg, normalized_boxes, classes_names, confidences):
    colors = list(ImageColor.colormap.values())
    image_np = np.array(Image.open(img_jpg))
    plt.figure(figsize=(20, 20))
    ax = plt.axes()
    ax.imshow(image_np)

    for idx in range(len(normalized_boxes)):
        left, bot, right, top = normalized_boxes[idx]
        if confidences[i] >= conf_threshold:
            x, w = [val * image_np.shape[1] for val in [left, right - left]]
            y, h = [val * image_np.shape[0] for val in [bot, top - bot]]
            color = colors[hash(classes_names[idx]) % len(colors)]
            rect = patches.Rectangle((x, y), w, h, linewidth=3, edgecolor=color, facecolor="none")
            ax.add_patch(rect)
            ax.text(
                x,
                y,
                "{} {:.0f}%".format(classes_names[idx], confidences[idx] * 100),
                bbox=dict(facecolor="white", alpha=0.5),
            )

In [ ]:
import os
import random

image_dir = os.path.join("data", "tiny_motorbike", "JPEGImages")
test_images = os.listdir(image_dir)
random.shuffle(test_images)

for i in range(3):
    test_image = os.path.join(image_dir, test_images[i])
    query_response = query(base_model_predictor, test_image)
    normalized_boxes, classes_names, confidences = parse_response(query_response)
    display_predictions(test_image, normalized_boxes, classes_names, confidences)

### Jumpstart를 활용하여 fine-tuning 진행

- 데이터의 포맷의 변경이 필요합니다.
- 받은 데이터의 폴더 구조 변경은 아래와 같이 필요합니다.
```
input_directory
    |--images
        |--abc.png
        |--def.png
    |--annotations.json
```
- 추가로 bbox format은 `COCO format`인데 `VOC format`으로 변경이 되어야 합니다.
- `COCO format` : [x, y, width, height]
- `VOC format` : [x_min, y_min, x_max, y_max]

In [ ]:
!pip install -q pybboxes

In [ ]:
import pybboxes as pbx
annotation_file = os.path.join("data", "pothole", "Annotations", "usersplit_train_cocoformat.json")
with open(annotation_file, 'r') as file:
    annotations = json.load(file)
    for img in annotations["images"]:
        only_name = img["file_name"].split("/")[1]
        img["file_name"] = only_name
        
    for bbox in annotations["annotations"]:
        new_bbox = pbx.convert_bbox(bbox["bbox"], from_type="coco", to_type="voc")
        # print(bbox["bbox"], list(new_bbox))
        bbox["bbox"] = new_bbox
        
result_file = os.path.join("data", "pothole", "annotations.json")
with open(result_file, 'w', encoding='utf-8') as file:
    json.dump(annotations, file)
    

- S3에 변경된 annotation 파일 업로드 및 폴더 이름 변경등을 진행하고 학습을 진행할 수 있습니다.
- 미리 준비된 fine-tuning 코드를 활용하여 학습을 진행합니다.

In [ ]:
from sagemaker import image_uris, model_uris, script_uris

# Currently, not all the object detection models in jumpstart support finetuning. Thus, we manually select a model
# which supports finetuning.
train_model_id, train_model_version, train_scope = (
    # "mxnet-od-ssd-512-vgg16-atrous-coco",
    "pytorch-od1-fasterrcnn-resnet50-fpn",
    "*",
    "training",
)
training_instance_type = "ml.p3.2xlarge"

# Retrieve the docker image
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    model_id=train_model_id,
    model_version=train_model_version,
    image_scope=train_scope,
    instance_type=training_instance_type,
)

# Retrieve the training script. This contains all the necessary files including data processing, model training etc.
train_source_uri = script_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, script_scope=train_scope
)
# Retrieve the pre-trained model tarball to further fine-tune
train_model_uri = model_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, model_scope=train_scope
)

print(train_image_uri)
print(train_source_uri)
print(train_model_uri)

In [ ]:
training_dataset_s3_path = f"s3://{bucket}/object_detection_data/pothole/"
s3_output_location = f"s3://{bucket}/jumpstart-od-finetuning/output"

In [ ]:
from sagemaker import hyperparameters

# Retrieve the default hyper-parameters for fine-tuning the model
hyperparameters = hyperparameters.retrieve_default(
    model_id=train_model_id, model_version=train_model_version
)

# [Optional] Override default hyperparameters with custom values
hyperparameters["epochs"] = "10"
print(hyperparameters)

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base
from sagemaker.tuner import HyperparameterTuner

training_job_name = name_from_base(f"jumpstart-od-{train_model_id}-transfer-learning")

# Create SageMaker Estimator instance
od_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",  # Entry-point file in source_dir and present in train_source_uri.
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    base_job_name=training_job_name,
)

In [ ]:
od_estimator.fit({"training": training_dataset_s3_path}, logs=True)

In [ ]:
inference_instance_type = "ml.p3.2xlarge"

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    image_scope="inference",
    model_id=train_model_id,
    model_version=train_model_version,
    instance_type=inference_instance_type,
)

# Retrieve the inference script uri. This includes scripts for model loading, inference handling etc.
deploy_source_uri = script_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, script_scope="inference"
)

endpoint_name = name_from_base(f"jumpstart-od-endpoint-{train_model_id}-")

# Use the estimator from the previous step to deploy to a SageMaker endpoint
predictor = od_estimator.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",  # entry point file in source_dir and present in deploy_source_uri
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name,
)

In [ ]:
image_dir = os.path.join("data", "pothole", "JPEGImages")
test_images = os.listdir(image_dir)
random.shuffle(test_images)

In [ ]:
from PIL import Image

for i in range(5):
    test_image = os.path.join(image_dir, test_images[i])
    Image.open(test_image).convert('RGB').save(test_image)  # alpha channel 있는 경우 에러 발생하므로 추가함. 
    query_response = query(predictor, test_image)
    normalized_boxes, classes_names, confidences = parse_response(query_response)
    display_predictions(test_image, normalized_boxes, classes_names, confidences)
